In [ ]:
import os
import pandas as pd # type: ignore
import numpy as np # type: ignore
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tqdm.notebook import tqdm
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, FloatVector


# Convert pandas.DataFrames to R dataframes automatically.
pandas2ri.activate()

try:
    os.chdir('/container/mount/point')
except FileNotFoundError:
    print("Warning: Directory '/container/mount/point' does not exist.")


from utils.helper import r_to_pandas, check_samples_overlap, generate_taxa_dict, calculate_obs_stat_and_pvalues
from utils.helper import perform_bh_correction_and_filter

In [ ]:
def calculate_unadjusted_p_values(test_stats, obs_stat, test_type):
    """
    Calculate unadjusted p-values for observed test statistics using permutation test results.

    Parameters
    ----------
    test_stats : pd.DataFrame
        DataFrame of test statistics from permutations (rows: taxa/features, columns: permutations).
    obs_stat : pd.Series or pd.DataFrame
        Observed test statistics (index: taxa/features).
    test_type : str
        Type of test to perform. Currently supports "two-sided".

    Returns
    -------
    p_value : pd.DataFrame
        DataFrame of unadjusted p-values (index: taxa/features, column: "p-value").
    """
    if test_type == "two-sided":
        # two-sided test with absolute value tests both directions
        # calculate the proportion of |T_rand| >= |T_obs|
        p_value = test_stats.abs().ge(obs_stat.abs().values, axis="rows")
        p_value.index = obs_stat.index
        p_value = p_value.mean(axis="columns").sort_values().to_frame()
        p_value.columns = ["p-value"]

    return p_value

In [ ]:
def min_unadjusted_p_values(test_stats):
    """
    Calculate the minimum unadjusted p-value for each permutation by treating each column as the observed statistic.

    For each permutation (column), treats its values as the observed statistics,
    compares against all other permutations, and computes the minimum p-value across all taxa/features.

    Parameters
    ----------
    test_stats : pd.DataFrame
        DataFrame of test statistics from permutations (rows: taxa/features, columns: permutations).

    Returns
    -------
    min_p_values : np.ndarray
        Array of minimum unadjusted p-values for each permutation.
    """
    min_p_values = []

    for i in test_stats.columns:
        # Choose random statistic as "observed" statistic
        obs_stat = test_stats.loc[:, i]

        # Select random statistics except the observed one
        t = test_stats.loc[:, test_stats.columns != i]

        # Select statistics greater or equal to the observed one
        t_comp = t.abs().ge(obs_stat.abs().values, axis="rows")

        # Calculate unadjusted p-values
        t_comp = t_comp.mean(axis="columns")

        # Save min p-value among all species
        min_p_values.append(min(t_comp))
        
    min_p_values = np.array(min_p_values)
    
    return min_p_values

In [ ]:
def adjusted_p_values(min_p_values, p_value):
    """
    Calculate adjusted p-values using the distribution of minimum unadjusted p-values.

    For each observed p-value, computes the proportion of min_p_values less than or equal to it.

    Parameters
    ----------
    min_p_values : np.ndarray or list
        Array or list of minimum unadjusted p-values from permutations.
    p_value : pd.DataFrame
        DataFrame of observed unadjusted p-values (column: 'p-value').

    Returns
    -------
    adj_p_values : list
        List of adjusted p-values for each observed statistic.
    """
    adj_p_values = []
    p = p_value.shape[0]

    for i in range(p):
        adj = np.mean(min_p_values <= p_value['p-value'][i])
        adj_p_values.append(adj)
    
    return adj_p_values

In [ ]:
def perform_bh_correction_and_filter(stats, alpha=0.05):
    """
    Perform Benjamini-Hochberg correction on p-values and filter significant features.

    Parameters:
    - taxa (DataFrame): DataFrame containing taxonomic information.
    - stats (DataFrame): DataFrame containing statistical results with 'p_value' column.
    - alpha (float, optional): Threshold for significance. Default is 0.05.

    Returns:
    - DataFrame: Subset of the input DataFrame containing features with adjusted p-values <= alpha.
    """
    # Perform Benjamini-Hochberg correction
    adjusted_pvalues = sm.stats.multipletests(stats['p_value'], method='fdr_bh')[1]

    # Update the DataFrame with adjusted p-values
    stats['adjusted_pvalue'] = adjusted_pvalues
    stats['effect'] = np.sign(stats['obs_stat'])

    # Display the result
    da_species = stats[stats['adjusted_pvalue'] <= alpha]

    return da_species

In [ ]:
utils = importr('utils')
devtools = importr('devtools')
linda = importr("LinDA")

### KORA Dataset

In [ ]:
# Load your  matched sample dataframe and ASV table
kora_matched_df = pd.read_csv("data/smoking_KORA_experiment.csv", index_col=0)
asv = pd.read_csv("data/filtered_count_table.csv", index_col=0)
simulated_outcomes = pd.read_csv("data/simulated_KORA_outcomes.csv", index_col=0)
taxa = pd.read_csv('data/taxonomy_clean.csv', index_col=0)
print(f"ASV table shape (features, samples): {asv.shape}")

# Sort ASV columns to match sample order in kora_matched_df
sample_order = list(kora_matched_df.index.astype(str))
ASV_table = asv.reindex(sample_order, axis=1)

taxa_dict = {}
for level in ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']:
    df_level = ASV_table.join(taxa[level])
    df_level = df_level.groupby(level).sum()
    taxa_dict[level] = df_level
taxa_dict["ASVs"] = ASV_table

for level in taxa_dict.keys():
    print(f"{level} count table shape: {taxa_dict[level].shape}")

# Overlap 16S and IgE samples
sample_ids = ASV_table.columns.astype(str)
matched_samples = kora_matched_df[kora_matched_df.index.astype(str).isin(sample_ids)]

# Example: create a covariate DataFrame (replace 'W' with your covariate column)
w = pd.DataFrame(matched_samples["W"].values, index=matched_samples.index, columns=["w"])

# Now you can run your downstream analysis, e.g. LinDA
linda_stats = calculate_obs_stat_and_pvalues(taxa_dict, w)
print("------------------------- LinDA is DONE ------------------------- \n")

In [ ]:
# --- Simulated outcomes ---
n_iter = 1000
alpha = 0.05
target_variable = "smoking_bin"  # Make sure this matches your context

for level, data in taxa_dict.items():
    p, N = data.shape
    if p < 20:
        continue

    print(f"Linda test for {level} level")

    test_stats = []
    pvalue_list = []
    W_frame = simulated_outcomes.iloc[:, -n_iter:]

    for i in tqdm(range(n_iter)):
        w_tmp = W_frame.iloc[:, i].to_frame(name="w")
        w_tmp.index = w_tmp.index.astype(str)
        w_tmp = w_tmp[w_tmp.index.isin(data.columns)]

        # apply linda
        lo_tmp = linda.linda(data, w_tmp, formula="~w", alpha=alpha, prev_cut=0, lib_cut=1)
        linda_out_tmp = r_to_pandas(lo_tmp.rx2("output"))
        out = linda_out_tmp['w']

        test_stats.append(out['stat'].values)
        pvalue_list.append(out['pvalue'].values)

    # Save results
    test_stats_df = pd.DataFrame(test_stats).T
    test_stats_df.to_csv(f'data/linda_{level}_{n_iter}_{target_variable}.csv')

    pvalue_df = pd.DataFrame(pvalue_list).T
    pvalue_df.to_csv(f'data/linda_pvalues_{level}_{n_iter}_{target_variable}.csv')

In [ ]:
n_iter = 1000
p_value_dict = {}

for level in taxa_dict.keys():
    file_path = f'data/linda_pvalues_{level}_{n_iter}_{target_variable}.csv'
    output_path = f'data/KORA/adj_linda_{level}_{n_iter}_{target_variable}.csv'

    if os.path.exists(file_path):
        print(f"p-value adjustment procedure for {level} level")

        obs_stat = linda_stats[level]['obs_stat']
        test_stats = pd.read_csv(file_path, index_col=0)

        # Step 1: Unadjusted p-values
        p_value = calculate_unadjusted_p_values(test_stats, obs_stat, test_type="two-sided")

        # Step 2: Randomized minimum p-values
        min_p_values = min_unadjusted_p_values(test_stats)

        # Step 3: Adjusted p-values
        p_value["Lee et al."] = adjusted_p_values(min_p_values, p_value)

        p_value_dict[level] = p_value

        p_value.to_csv(output_path)

In [ ]:
adjusted_results_clusters = dict()
FDR_rate = 0.2

# Perform BH correction and filter results
result = perform_bh_correction_and_filter(linda_stats['ASV'], alpha=FDR_rate)

# Merge with taxonomy to include names
result_with_taxa = result.merge(taxa, left_index=True, right_index=True, how='left')

# Create a short taxon name for plotting
result_with_taxa['short_name'] = (
    result_with_taxa['family'].fillna('') + ' ' +
    result_with_taxa['genus'].fillna('') + ' ' +
    result_with_taxa['species'].fillna('')
).str.strip()

# Sort by adjusted p-value
result_with_taxa_sorted = result_with_taxa.sort_values('adjusted_pvalue')

# Bar plot for adjusted p-values
plt.figure(figsize=(10, 8))
plt.barh(result_with_taxa_sorted['short_name'], result_with_taxa_sorted['adjusted_pvalue'], color='skyblue')
plt.xlabel('Adjusted p-value')
plt.ylabel('Family Genus Species')
plt.title('Adjusted p-values for Differentially Abundant Species')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()